In [1]:
import os
from google.colab import userdata
os.environ["OPENAI_API_KEY"] = userdata.get('OPENAI_API_KEY')


In [3]:
!pip install -U langchain langchain-openai


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 23.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.4/62.4 kB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 434.1/434.1 kB 41.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 78.9 MB/s eta 0:00:00
  Attempting uninstall: langchain-core
    Found existing installation: langchain-core 0.3.52
    Uninstalling langchain-core-0.3.52:
      Successfully uninstalled langchain-core-0.3.52
  Attempting uninstall: langchain
    Found existing installation: langchain 0.3.23
    Uninstalling langchain-0.3.23:
      Successfully uninstalled langchain-0.3.23


In [4]:
from langchain_openai import OpenAIEmbeddings
embeddings = OpenAIEmbeddings()

In [11]:
!pip install -U langchain-community pypdf

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.3/302.3 kB 7.5 MB/s eta 0:00:00


In [12]:
from langchain.document_loaders import PyPDFLoader
loader = PyPDFLoader("/content/rag.pdf")
documents = loader.load()

In [13]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
documents = text_splitter.split_documents(documents)

In [14]:
 !pip install --q faiss-cpu

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 30.7/30.7 MB 85.9 MB/s eta 0:00:00


In [15]:
from langchain.vectorstores import FAISS
vectorstore = FAISS.from_documents(documents, embeddings)

In [16]:
retriever = vectorstore.as_retriever()

In [17]:
from langchain_openai import ChatOpenAI
llm = ChatOpenAI()

In [18]:
from langchain.prompts import ChatPromptTemplate
from langchain.schema.runnable import RunnablePassthrough
from langchain.schema.output_parser import StrOutputParser

template = """"
You are a helpful assistant that answers questions based on the provided context.
Use the provided context to answer the question.
Question: {input}
Context: {context}
Answer:
"""
prompt = ChatPromptTemplate.from_template(template)

# Setup RAG pipeline
rag_chain = (
    {"context": retriever,  "input": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

In [19]:
response = rag_chain.invoke("what is rag?")
response

'RAG stands for Retrieval-Augmented Generation, which integrates the capabilities of large language models (LLMs) with external knowledge retrieval to advance text generation tasks in natural language processing (NLP).'

In [21]:
# create dataset
question = ["what is rag?"]
response = []
contexts = []

# Inference
for query in question:
  response.append(rag_chain.invoke(query))
  contexts.append([docs.page_content for docs in retriever.get_relevant_documents(query)])

# To dict
data = {
    "query": question,
    "response": response,
    "context": contexts,
}

In [23]:
!pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.2/491.2 kB 12.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 12.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 13.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 12.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 14.6 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.2
    Uninstalling fsspec-2025.3.2:
      Successfully uninstalled fsspec-2025.3.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2025.3.2 requires fsspec==2025.3.2, but you have fsspec 2024.12.0 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which

In [24]:
from datasets import Dataset
dataset = Dataset.from_dict(data)

In [25]:
import pandas as pd
df = pd.DataFrame(dataset)

In [26]:
df

,query,response,context
0,what is rag?,"RAG stands for Retrieval-Augmented Generation,...",[7 RAG EVALUATION AND BENCHMARK\nRetrieval-Aug...


In [27]:
df_dict = df.to_dict(orient='records')

# Convert context to list
for record in df_dict:
    if not isinstance(record.get('contexts'), list):
        if record.get('contexts') is None:
            record['contexts'] = []
        else:
            record['contexts'] = [record['contexts']]

In [37]:
!pip install ragas


In [38]:
from datasets import Dataset

data = {
    "question": ["What is RAG?"],
    "contexts": [[
        "RAG (Retrieval-Augmented Generation) is a method that combines retrieval-based and generation-based models. It retrieves relevant documents and then uses them to generate more accurate and grounded responses."
    ]],
    "answer": ["RAG is a method that retrieves documents and then generates responses based on them."],
    "ground_truth": ["RAG is a method that combines document retrieval with text generation to improve accuracy and grounding in responses."]
}

dataset = Dataset.from_dict(data)


In [39]:
from ragas.evaluation import evaluate
from ragas.metrics import (
    faithfulness,
    answer_relevancy,
    context_recall,
    context_precision
)

result = evaluate(
    dataset=dataset,
    metrics=[
        context_precision,
        context_recall,
        faithfulness,
        answer_relevancy,
    ],
)

print(result)


Evaluating:   0%|          | 0/4 [00:00<?, ?it/s]

{'context_precision': 1.0000, 'context_recall': 1.0000, 'faithfulness': 1.0000, 'answer_relevancy': 1.0000}
